In [9]:
import os
import os.path as osp
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
pd.set_option('display.max_rows', 500)

In [10]:
files_35 = glob.glob(f"labels_35/*.txt")
files_70 = glob.glob(f"labels_70/*.txt")
files_80 = glob.glob(f"labels_80/*.txt")

In [13]:
def del_duplicate(text):
    text += "@"
    out = ""
    for i in range(len(text)-1):
        if text[i] != text[i+1]:
            out += text[i]
    return out

In [14]:
def post_process(files, th):
    df = pd.DataFrame(columns=["file", "label"])
    for file in tqdm(files):
        data = pd.read_csv(file, header=None, sep=" ", names=["label", "x1", "y1", "x2", "y2"])
        data["label"] = data["label"].astype("str")
        data = data.sort_values("x1")
        if data["y1"].std() > 10 or len(data) < 4:
            ans = ""
        else:
            data["label"][data["x1"].diff() > th] = " " + data["label"]
            ans = "".join(list(data["label"]))
            final_ans = ["", "", ""]
            tmp = ans.split(" ")
            for i, ele in enumerate(tmp):
                if i >= 3:
                    break
                if 1950 < int(ele) < 2020:
                    final_ans[i] = str(int(ele))
                    continue
                if len(ele) >= 3:
                    final_ans[i] = del_duplicate(ele)
                    if len(final_ans[i]) >= 5 and i == 0:
                        final_ans[i] = final_ans[i][:4]
                    if len(final_ans[i]) >= 3 and i != 0:
                        final_ans[i] = final_ans[i][1:]
                    if 100 < int(final_ans[i]) < 1000 and i==0:
                        final_ans[i] = final_ans[i][:2]
                    continue

                final_ans[i] = ele
            if final_ans[1] == "":
                final_ans[1] = final_ans[0][4:6]
                final_ans[2] = final_ans[0][-2:]
                final_ans[0] = final_ans[0][:4]
            elif final_ans[2] == "":
                final_ans[2] = final_ans[1][-2:]
                final_ans[1] = final_ans[1][:-2]
        if ans != "":
            ans =  " ".join(final_ans)
        file = osp.basename(file)
        file = file.replace("txt", "jpg")
        row = pd.Series([file, ans], index=df.columns)
        df = df.append(row, ignore_index=True)
    return df

In [15]:
def add_is_date(df):
    df["is_date"] = False
    for i, label in enumerate(df["label"]):
        if label != "":
            try:
                pd.to_datetime(df.loc[i, "label"], format="%Y %m %d")
                df.loc[i, "is_date"] = True
            except ValueError:
                pass
            try:
                pd.to_datetime(df.loc[i, "label"], format="%y %m %d")
                df.loc[i, "is_date"] = True
            except ValueError:
                pass
            try:
                pd.to_datetime(df.loc[i, "label"], format="%d %m %y")
                df.loc[i, "is_date"] = True
            except ValueError:
                pass
            try:
                pd.to_datetime(df.loc[i, "label"], format="%m %d %y")
                df.loc[i, "is_date"] = True
            except ValueError:
                pass
        else:
            df.loc[i, "is_date"] = True

In [16]:
def joinDf(df1, df2):
    for i in df1[df1["is_date"] == False].index:
        if df2.loc[i, "is_date"] == True:
            df1.loc[i, "label"] = df2.loc[i, "label"]
            df1.loc[i, "is_date"] = True
    print(len(df1[df1["is_date"] == True]))

In [17]:
def makeDf(files):
    df1 = post_process(files, 35)
    add_is_date(df1)
    df2 = post_process(files, 45)
    add_is_date(df2)
    return df1, df2

In [18]:
df_70_1, df_70_2 = makeDf(files_70)
joinDf(df_70_1, df_70_2)
df_80_1, df_80_2 = makeDf(files_80)
joinDf(df_70_1, df_80_1)
joinDf(df_70_1, df_80_2)
df_35_1, df_35_2 = makeDf(files_35)
joinDf(df_70_1, df_35_1)
joinDf(df_70_1, df_35_2)

C:\Users\kaigan1917\Miniconda3\envs\kaggle\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.



9610



9762
9766



9812
9813


In [25]:
df_70_1.drop("is_date")
df_70_1.to_csv("submission.csv", header=False, index=False)

KeyError: "['is_date'] not found in axis"

In [24]:
df_70_1

,file,label,is_date
0,test_id0001_no0001.jpg,94 8 9,True
1,test_id0001_no0002.jpg,,True
2,test_id0001_no0003.jpg,94 8 9,True
3,test_id0001_no0004.jpg,94 8 9,True
4,test_id0001_no0005.jpg,94 8 9,True
...,...,...,...
9902,test_id0071_no0044.jpg,,True
9903,test_id0071_no0045.jpg,,True
9904,test_id0071_no0046.jpg,,True
9905,test_id0071_no0047.jpg,,True
